In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df= pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2010-2011")

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


In [4]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [5]:
df.columns=[col.lower().replace(" ", "") for col in df.columns]

In [6]:
df.head()

,invoice,stockcode,description,quantity,invoicedate,price,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


In [7]:
df[df["invoice"].str.contains("C", na=False)].shape

(9288, 8)

In [8]:
df=df[~df["invoice"].str.contains("C", na=False)]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532622 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   invoice      532622 non-null  object        
 1   stockcode    532622 non-null  object        
 2   description  531168 non-null  object        
 3   quantity     532622 non-null  int64         
 4   invoicedate  532622 non-null  datetime64[ns]
 5   price        532622 non-null  float64       
 6   customerid   397925 non-null  float64       
 7   country      532622 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.6+ MB


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,532622.00000,10.23995,159.59340,-9600.00000,1.00000,3.00000,10.00000,80995.00000
price,532622.00000,3.84765,41.75799,-11062.06000,1.25000,2.08000,4.13000,13541.33000
customerid,397925.00000,15294.30860,1713.17274,12346.00000,13969.00000,15159.00000,16795.00000,18287.00000


In [11]:
df=df[df["quantity"]>0]

In [12]:
df.dropna(inplace=True)

In [13]:
df.isnull().sum()

invoice        0
stockcode      0
description    0
quantity       0
invoicedate    0
price          0
customerid     0
country        0
dtype: int64

In [14]:
df["total_price"] = df["price"]*df["quantity"]

In [15]:
df.head()

,invoice,stockcode,description,quantity,invoicedate,price,customerid,country,total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom,15.30000
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom,22.00000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000


In [20]:
cltv=df.groupby("customerid").agg({"invoice": lambda x: x.nunique(),"quantity": lambda x: x.sum(), "total_price": lambda x: x.sum()})

In [22]:
cltv.sample(10)

,invoice,quantity,total_price
customerid,,,
15045.00000,2,114,735.66000
14946.00000,1,22,100.90000
15604.00000,1,159,216.39000
14714.00000,1,229,160.19000
13428.00000,1,156,218.47000
17232.00000,2,203,421.52000
14682.00000,2,80,52.00000
15147.00000,1,366,623.50000
16796.00000,1,186,305.62000


In [23]:
cltv.columns= ["total_transaction", "total_unit","total_price"]

In [24]:
cltv.head()

,total_transaction,total_unit,total_price
customerid,,,
12346.00000,1,74215,77183.60000
12347.00000,7,2458,4310.00000
12348.00000,4,2341,1797.24000
12349.00000,1,631,1757.55000
12350.00000,1,197,334.40000


In [25]:
# average order value (average_order_value = total_price / total_transaction)

In [30]:
cltv["average_order_value"] = cltv["total_price"] / cltv["total_transaction"]

In [31]:
cltv.head()

,total_transaction,total_unit,total_price,average_order_value
customerid,,,,
12346.00000,1,74215,77183.60000,77183.60000
12347.00000,7,2458,4310.00000,615.71429
12348.00000,4,2341,1797.24000,449.31000
12349.00000,1,631,1757.55000,1757.55000
12350.00000,1,197,334.40000,334.40000


In [32]:
# Purchase Frequency (total_transaction / total_number_of_customers)

In [33]:
cltv.shape[0]

4339

In [34]:
cltv["purchase_frequency"] = cltv["total_transaction"] / cltv.shape[0]

In [35]:
# Repeat rate and churn rate

In [38]:
repeat_rate=cltv[cltv["total_transaction"]>1].shape[0] / cltv.shape[0]

In [39]:
churn_rate=1-repeat_rate

In [40]:
# Profit Margin (profit_margin =  total_price * 0.10)

In [41]:
cltv["profit_margin"] = cltv["total_price"]*0.10

In [42]:
# Customer Value (customer_value = average_order_value * purchase_frequency)

In [43]:
cltv["customer_value"] = cltv["average_order_value"] * cltv["purchase_frequency"]

In [44]:
# Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)

In [45]:
cltv["cltv"]=(cltv["customer_value"]/churn_rate)*cltv["profit_margin"]

In [46]:
cltv.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
customerid,,,,,,,,
12346.00000,1,74215,77183.60000,77183.60000,0.00023,7718.36000,17.78834,398748.86941
12347.00000,7,2458,4310.00000,615.71429,0.00161,431.00000,0.99332,1243.38019
12348.00000,4,2341,1797.24000,449.31000,0.00092,179.72400,0.41421,216.20292
12349.00000,1,631,1757.55000,1757.55000,0.00023,175.75500,0.40506,206.75917
12350.00000,1,197,334.40000,334.40000,0.00023,33.44000,0.07707,7.48483


In [47]:
cltv.sort_values("cltv", ascending=False)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
customerid,,,,,,,,
14646.00000,74,197491,280206.02000,3786.56784,0.01705,28020.60200,64.57848,5255382.43937
18102.00000,60,64124,259657.30000,4327.62167,0.01383,25965.73000,59.84266,4512845.61200
17450.00000,46,69993,194550.79000,4229.36500,0.01060,19455.07900,44.83770,2533467.86410
16446.00000,2,80997,168472.50000,84236.25000,0.00046,16847.25000,38.82749,1899798.07605
14911.00000,201,80515,143825.06000,715.54756,0.04632,14382.50600,33.14705,1384581.51834
...,...,...,...,...,...,...,...,...
17956.00000,1,1,12.75000,12.75000,0.00023,1.27500,0.00294,0.01088
16454.00000,2,3,6.90000,3.45000,0.00046,0.69000,0.00159,0.00319
14792.00000,1,2,6.20000,6.20000,0.00023,0.62000,0.00143,0.00257


In [48]:
# Segmentation

In [49]:
cltv["segment"] = pd.qcut(cltv["cltv"],4, labels=["D", "C", "B", "A"])

In [50]:
cltv.sort_values("cltv", ascending=False)

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
customerid,,,,,,,,,
14646.00000,74,197491,280206.02000,3786.56784,0.01705,28020.60200,64.57848,5255382.43937,A
18102.00000,60,64124,259657.30000,4327.62167,0.01383,25965.73000,59.84266,4512845.61200,A
17450.00000,46,69993,194550.79000,4229.36500,0.01060,19455.07900,44.83770,2533467.86410,A
16446.00000,2,80997,168472.50000,84236.25000,0.00046,16847.25000,38.82749,1899798.07605,A
14911.00000,201,80515,143825.06000,715.54756,0.04632,14382.50600,33.14705,1384581.51834,A
...,...,...,...,...,...,...,...,...,...
17956.00000,1,1,12.75000,12.75000,0.00023,1.27500,0.00294,0.01088,D
16454.00000,2,3,6.90000,3.45000,0.00046,0.69000,0.00159,0.00319,D
14792.00000,1,2,6.20000,6.20000,0.00023,0.62000,0.00143,0.00257,D


In [52]:
cltv.groupby("segment").agg({"mean","count", "sum"})

total_transaction              total_unit                total_price  \
                     mean    sum count       mean      sum count        mean   
segment                                                                        
D                 1.26544   1373  1085  129.48664   140493  1085   178.92174   
C                 1.87558   2035  1085  286.29677   310632  1085   464.36576   
B                 3.51015   3805  1084  649.44188   703995  1084  1071.87575   
A                10.43594  11323  1085 3711.13088  4026577  1085  6499.12042   

                            average_order_value                     \
                  sum count                mean          sum count   
segment                                                              
D        194130.09000  1085           154.89625 168062.43060  1085   
C        503836.85200  1085           303.78336 329604.94783  1085   
B       1161913.31100  1084           398.16218 431607.80516  1084   
A       7051545.65100  1085           819.33391 888977.29343  1085   

        purchase_frequency               profit_margin                     \
                      mean     sum count          mean          sum count   
segment                                                                     
D                  0.00029 0.31643  1085      17.89217  19413.00900  1085   
C                  0.00043 0.46900  1085      46.43658  50383.68520  1085   
B                  0.00081 0.87693  1084     107.18757 116191.33110  1084   
A                  0.00241 2.60959  1085     649.91204 705154.56510  1085   

        customer_value                         cltv                       
                  mean        sum count        mean            sum count  
segment                                                                   
D              0.04124   44.74074  1085     2.49687     2709.10807  1085  
C              0.10702  116.11820  1085    15.24582    16541.70928  1085  
B              0.24703  267.78366  1084    82.22465    89131.51565  1084  
A              1.49784 1625.15456  1085 22649.36689 24574563.07585  1085

In [53]:
cltv.to_csv("CLTV.csv")